## Tools for processing platelet picklists to Hamilton genotyping picklists

In [1]:
import csv
import os
import pandas as pd
import pickle
import re
import string

from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

import cauldron_sdk as api
from Genotyping_picklist import make_picklist

%load_ext autoreload
%autoreload 2

### User input, tweak next cell as appropriate

In [2]:
# path is the location of the picklists
# picklist_files are the platelet picklist files
path = '../CRISPR_genotyping/IB-GP-E3-H/'
picklist_files = ['IB-GP-E3-H-B1_IB-PG-E1-H-B3_guide_picklist.csv',
                  'IB-GP-E3-H-B2_IB-GP-E2-H-B3_guide_picklist.csv',
                  'IB-GP-E3-H-B3_guide_picklist.csv']

In [3]:
experiments = {'_'.join(i.split('_')[:-1]):{'picklist_file':os.path.join(path,i)} for i in picklist_files}

In [4]:
for experiment in experiments:
    exp_dir = os.path.join(path,experiment)
    if not os.path.isdir(exp_dir): os.mkdir(exp_dir)
    experiments[experiment]['directory'] = exp_dir

## Functions and variables for turning platelet picklists into 2d maps and Hamilton picklists

In [5]:
PCR_suffix = '_hamilton_PCR_picklist.csv'
seq_suffix = '_hamilton_seq_picklist.csv'

def add_row_col(long_skinny):
    # df, reads the 'Destination Well' field and adds row and column fields from that
    rows = []
    cols = []
    for i,line in long_skinny.iterrows():
        well = line['Destination Well']
        match = re.match(r"([a-z]+)([0-9]+)", well, re.I)
        row,col = match.groups()
        rows.append(row)
        cols.append(int(col))
    long_skinny['row'] = rows
    long_skinny['column'] = cols
    return

def build_outmaps(experiments):
    # takes a list of dfs and builds a dict of 2D dfs of the individual quadrants
    # Blank 384 well plate
    r = list(string.ascii_uppercase)[:16]
    c = [i for i in range(1,25)]
    blank_plate = pd.DataFrame(columns = c, index = r)   
    old_column_names = list(range(13,25))
    COL_RENAME = {}
    for i,j in zip(c, old_column_names):
        COL_RENAME[j]=i
    old_row_names = list(string.ascii_uppercase)[8:16]
    ROW_RENAME = {}
    for i,j in zip(old_row_names, r):
        ROW_RENAME[i] = j

    for experiment in experiments:
        picklist = experiments[experiment]['picklist']
        plates = picklist.Destination.unique()
        PLATES = {}
        for plate in plates:
            QUADS = {}
            platemap = blank_plate.copy()
            temp_df = picklist[picklist['Destination'] == plate]
            for idx,line in temp_df.iterrows():
                row = line['row']
                column = line['column']
                rec = line['REC ID']
                platemap.loc[row,column] = rec
            Q1 = platemap.iloc[:8,:12]
            Q2 = platemap.iloc[:8,12:]
            Q3 = platemap.iloc[8:,:12]
            Q4 = platemap.iloc[8:,12:]
            Q2.rename(columns = COL_RENAME, inplace = True)
            Q3.rename(index = ROW_RENAME, inplace = True)
            Q4.rename(index = ROW_RENAME, columns = COL_RENAME, inplace = True)
            for i, j in zip(['Q1','Q2','Q3','Q4'],[Q1,Q2,Q3,Q4]):
                if not j.dropna().empty: QUADS[i]=j   
            MAPS = {'quad_maps':QUADS}    
            PLATES[plate] = MAPS   
        experiments[experiment]['plates'] = PLATES
    return

def platelet_to_dict(experiments):
    # Takes a lists of platelet csv files and creates a list of DFs with the pertinent information
    fields = ['Destination','Destination Well','REC ID']
    for experiment in experiments:
        data_file = experiments[experiment]['picklist_file']
        temp = pd.read_csv(data_file)
        temp = temp[fields]
        add_row_col(temp)
        experiments[experiment]['picklist'] = temp
    return

def write_maps(experiments):
    engine = 'xlsxwriter'  # must have installed
    for experiment in experiments:
        for plate in experiments[experiment]['plates']:
            out_file = f'{plate}_map.xlsx'
            out_dir = experiments[experiment]['directory']
            with pd.ExcelWriter(os.path.join(out_dir,out_file), engine=engine) as writer:
                for i in experiments[experiment]['plates'][plate]['quad_maps']:
                    experiments[experiment]['plates'][plate]['quad_maps'][i].to_excel(writer, sheet_name=i)
    return

### Read the picklists and reformat to 2D quadrant maps. Save the maps

In [6]:
platelet_to_dict(experiments)
build_outmaps(experiments)
write_maps(experiments)

### Functions to interact with the gSheet "HTS Primer Master List"

In [7]:
# NOTE -change paths in get_creds to reference autoprimer primary directory
PRIMER_MASTERLIST_SPREADSHEET_ID = '1l2F7wasrVrYSHFIfX0sN2SFkzFz46CCHJfIBXb5R0n4'
PRIMERS_SHEETNAME = 'Primers'
PRIMER_MIXES_SHEETNAME = 'PM and sample name'

SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']

def fetch_data(spreadsheet_id, sheet, data_range):
    creds = get_creds()
    service = build('sheets', 'v4', credentials=creds)
    sheet_range_str = f'{sheet}!{data_range}'
    sheet = service.spreadsheets()
    result = sheet.values().get(spreadsheetId=spreadsheet_id,
                                range=sheet_range_str).execute()
    values = result.get('values', [])
    return values

def fetch_primers_data():
    return primer_masterlist_sheet2df(PRIMERS_SHEETNAME, 'A1:K', 'CO#')


def fetch_primer_mixes_data():
    primer_mixes_df = primer_masterlist_sheet2df(PRIMER_MIXES_SHEETNAME,
                                                 'A1:O',
                                                 'GRNA REC_ID').dropna(how='all')
    return primer_mixes_df

def get_creds():
    creds = None
    # The file token.pickle stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    if os.path.exists('../token.pickle'):
        with open('../token.pickle', 'rb') as token:
            creds = pickle.load(token)
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)#/Users/chris.johnson/.config/gcloud/application_default_credentials.json
            creds = flow.run_local_server()
        # Save the credentials for the next run
        with open('../token.pickle', 'wb') as token:
            pickle.dump(creds, token)
    return creds

def primer_masterlist_sheet2df(sheetname, data_range, index_name):
    data = fetch_data(PRIMER_MASTERLIST_SPREADSHEET_ID, sheetname, data_range)
    df = pd.DataFrame.from_records(data[1:], columns=data[0], index=index_name)
    return df

### Test by REC-ID to see if guides in the picklist are in the primer master list gSheet

In [8]:
# Identify any missing REC_IDs
selected_primer_mixes_df = fetch_primer_mixes_data()
available_gRNA = set(selected_primer_mixes_df.index)

missing_REC_IDs = set()
for experiment in experiments:
    picklist = experiments[experiment]['picklist']
    gRNAs = set(picklist['REC ID'])
    not_there = gRNAs.difference(available_gRNA)
    missing_REC_IDs = (missing_REC_IDs | not_there)
print('missing',len(missing_REC_IDs),'gRNA REC_IDs in Primer masterlist gSheet')

missing 0 gRNA REC_IDs in Primer masterlist gSheet


In [9]:
# This reports any gRNA REC-IDs not found in Sarah's sheet.
# If any are reported, they need to be resolved before proceeding
missing_REC_IDs

set()

## Run genotyping picklist maker
use the make_picklist function imported from Genoytping_picklist.py

In [ ]:
# If you wish to use only certain plates for primer mixes (mix_plates) or sequencing primers (seq_plates)
# definee that here
mix_plates = '26,27,28,29'
seq_plates = '44,45'

In [13]:
# Run the picklist making function for each quad in each plate in each experiment
# Verbose output so if an error happens it may be possible to see where
for experiment in experiments:
    print('Experiment',experiment)
    directory = experiments[experiment]['directory']
    for plate in experiments[experiment]['plates']:
        print('Plate',plate)
        PCR_picklists = []
        seq_picklists = []
        map_file = os.path.join(directory,plate+'_map.xlsx')
        for quad in experiments[experiment]['plates'][plate]['quad_maps']:
            print('Quad',quad)
            # make_picklist(map_file, quad) # Use any and all primer plates
            make_picklist(map_file, quad, mix_plates=mix_plates, seq_plates=seq_plates) # Use specific primer plates
            base_name = '.'.join(map_file.split('.')[:-1])
            PCR_picklists.append(f'{base_name}_{quad}{PCR_suffix}')
            seq_picklists.append(f'{base_name}_{quad}{seq_suffix}')
        experiments[experiment]['plates'][plate]['PCR picklists'] = PCR_picklists
        experiments[experiment]['plates'][plate]['seq picklists'] = seq_picklists

Experiment IB-GP-E3-H-B1_IB-PG-E1-H-B3_guide
Plate IB-GP-E3-H-B1_IB-PG-E1-H-B3 P-1
Quad Q1
Quad Q2
Quad Q3
Quad Q4
Experiment IB-GP-E3-H-B2_IB-GP-E2-H-B3_guide
Plate IB-GP-E3-H-B2_IB-GP-E2-H-B3 P-1
Quad Q1
Quad Q2
Quad Q3
Quad Q4
Experiment IB-GP-E3-H-B3_guide
Plate IB-GP-E3-H-B3 P-1
Quad Q1
Quad Q3


## File handling, list making after using scripts to make Hamilton picklist

### Functions to combine hamilton picklists and create name files

In [14]:
def update_locations(df_in, sfx):
    # incoming DF should have columns 'Labware_ID' and 'Dest_ID' -will update certain entries with quadrant number
    df = df_in.copy()
    sfx = str(sfx)
    columns = df.columns.tolist()
    labware = columns.index('Labware_ID')
    destination = columns.index('Dest_ID')
    for i in range(len(df)):
        labware_id = str(df.iloc[i,labware])
        dest_id = str(df.iloc[i,destination])
        if 'Plate' in labware_id:
            df.iloc[i,labware] = 'Plate'+sfx
        if 'Destination' in dest_id:
            df.iloc[i,destination] = 'Destination'+sfx
    return df 

def concat_picklists(path, picklists):
    # General form for reading, updating and concatanating picklists
    picklists_all = []
    for picklist_file in picklists:
        picklist = pd.read_csv(picklist_file)
        quad = picklist_file.split('_')[-4][-1] # read quadrant from file name
        picklists_all.append(update_locations(picklist, quad))
    return pd.concat(picklists_all)

def combine_picklists(PCR_picklist, seq_picklist):
    # combines info from the PCR picklist and seq picklist
    REC_ID_df = PCR_picklist[PCR_picklist['Source_plate'] != PCR_picklist['Source_plate']].copy()
    PM_df = PCR_picklist[PCR_picklist['Source_plate'] == PCR_picklist['Source_plate']].copy()
    REC_ID_df.set_index(['Dest_ID','Dest_Pos'], inplace = True)
    PM_df.set_index(['Dest_ID','Dest_Pos'], inplace = True)
    
    drop_cols = ['Position', 'Labware_ID', 'Source_plate']
    PM_df = PM_df.drop(columns = drop_cols)
    REC_ID_df = REC_ID_df.drop(columns = drop_cols)
    PM_df.rename(columns = {'Additional_information':'Primer_Mix'}, inplace = True)
    REC_ID_df.rename(columns = {'Additional_information':'REC_ID'}, inplace = True)
    seq_df = pd.merge(PM_df, REC_ID_df, left_index = True, right_index = True)
    
    seq_picklist = seq_picklist.set_index(['Dest_ID','Dest_Pos']).copy()
    seq_picklist = seq_picklist[['sequencing_primer']]
    
    return pd.merge(seq_df, seq_picklist, left_index = True, right_index = True)

def add_genes(seq_df, REC_IDS, SEQUENCES):
    # looks up information of REC_IDs in cauldron
    temp = seq_df.copy()
    ids = temp['REC_ID'].unique().tolist()
    genes = []
    sequences = []
    for ID in ids: # Add things that aren't in the dicts yet
        if ID not in REC_IDS:
            x = [i for i in api.guides.find(rec_ids = ID)][0]
            REC_IDS[ID] = x['gene']
            SEQUENCES[ID] = x['target_sequence']

    rec_idx = temp.columns.tolist().index('REC_ID')
    for i in range(len(temp)):
        rec_id = seq_df.iloc[i, rec_idx]
        genes.append(REC_IDS[rec_id])
        sequences.append(SEQUENCES[rec_id])
    temp['Gene'] = genes
    temp['gRNA_Sequence'] = sequences
    return temp, REC_IDS, SEQUENCES

def add_names(df, exp, plt):
    # adds columns with desired naming conventions to df
    seq_df = df.copy()
    genewiz_names = []
    synthego_names = []
    control_names = []
    for i in range(len(seq_df)):
        row = seq_df.iloc[i]
        gene = row['Gene']
        grna = '-'.join(row['REC_ID'].split('-')[1:])
        q = 'Q'+str(row.name[0][-1])
        pos = row.name[1]
        pm = row['Primer_Mix']
        seq_primer = row['sequencing_primer']
        genewiz_name = '-'.join([gene,grna,exp,plt,q,pos,pm]) # moved ,pos from before exp to after q
        synthego_name = genewiz_name+'-'+seq_primer+'.ab1'
        ctrl_expt = exp[:-1]+'0'
        control_name = ('-'.join([gene,grna,ctrl_expt,'P0','Q0','WT',pm,seq_primer]))+'.ab1'
        genewiz_names.append(genewiz_name)
        synthego_names.append(synthego_name)
        control_names.append(control_name)
    seq_df['Genewiz_name'] = genewiz_names
    seq_df['Synthego_name'] = synthego_names
    seq_df['Control_name'] = control_names
    return seq_df

def add_placeholders(seq_df):
    # Adds any missing wells as empty rows in the df (assumes 96 well format for all listed plates)
    r = list(string.ascii_uppercase)[:8]                       # rows in a full plate
    c = [i for i in range(1,13)]                               # columns in a full plate
    well = [i+str(n) for n in c for i in r]                    # wells in a full plate
    dest = seq_df.index.get_level_values(0).unique().tolist()  # destination plates in the passed df
    temp = [[d,w,0] for d in dest for w in well]               # list of all wells in full plates in the passed df
    temp_df = pd.DataFrame(temp, columns = ['Dest_ID','Dest_Pos','mock']).set_index(['Dest_ID','Dest_Pos'])
    return pd.merge(temp_df,seq_df, left_index = True, right_index = True, how = 'left').drop(columns = ['mock'])

### Combine picklists and make names
This can take a while beccaues of the need to query cauldron in the add_genes function

In [15]:
REC_IDS = {}
SEQUENCES = {}
for experiment in experiments:
    exp_dir = experiments[experiment]['directory']
    picklist_dir = os.path.join(exp_dir,'picklists')
    if not os.path.isdir(picklist_dir): os.mkdir(picklist_dir)
    plates = experiments[experiment]['plates']
    for plate in plates:
        plt = plate.split(' ')[-1]
        PCR_picklist_concat = concat_picklists(exp_dir, plates[plate]['PCR picklists'])
        seq_picklist_concat = concat_picklists(exp_dir, plates[plate]['seq picklists'])
        seq_name = f'{plate}_map_all{seq_suffix}' 
        seq_picklist_concat.to_csv(os.path.join(picklist_dir,seq_name), index = False)
        PCR_name = f'{plate}_map_all{PCR_suffix}'
        PCR_picklist_concat.to_csv(os.path.join(picklist_dir,PCR_name), index = False)
        
        out_df = combine_picklists(PCR_picklist_concat, seq_picklist_concat)

        out_df, REC_IDS, SEQUENCES = add_genes(out_df, REC_IDS, SEQUENCES)
        out_df = add_names(out_df, experiment, plt)
        out_df = add_placeholders(out_df)
        out_name = f'{plate}_sample_names.csv'
        out_df.to_csv(os.path.join(picklist_dir,out_name))